In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import pulp
from pulp.apis.glpk_api import GLPK_CMD
os.path.abspath(os.getcwd())

'd:\\GIT_REPOS\\2021-NCKU_ORA\\ASGMT_04'

# Data Envelopment Analysis
> Problem 3

## Read Data
#### `ORA_Assignment_04_DEA.xlsx`

In [2]:
df = pd.read_excel('ORA_Assignment_04_DEA.xlsx')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   School                47 non-null     object 
 1   Department            41 non-null     object 
 2   Personnel             41 non-null     float64
 3   Expenses (unit:1000)  41 non-null     float64
 4   Space                 41 non-null     float64
 5   Teaching              41 non-null     float64
 6   Publications          41 non-null     float64
 7   Grants (unit:1000)    41 non-null     float64
dtypes: float64(6), object(2)
memory usage: 3.1+ KB


,School,Department,Personnel,Expenses (unit:1000),Space,Teaching,Publications,Grants (unit:1000)
0,Liberal Arts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Chinese,33.5,6079.5,5376.0,26131.0,5.0,4250.9
2,2,Foreign Languages,36.5,8387.4,4574.0,40105.0,0.0,1543.5
3,3,History,26.0,5104.8,3795.0,13678.0,0.0,1729.1
4,4,Arts,6.0,3420.6,1610.0,4926.0,0.0,3562.2


In [3]:
s_type = np.append(np.full(4, 'Liberal Arts'), np.full(6, 'Sciences'))
s_type = np.append(s_type, np.full(18, 'Engineering'))
s_type = np.append(s_type, np.full(5, 'Management'))
s_type = np.append(s_type, np.full(5,'Medicine'))
s_type = np.append(s_type, np.full(3, 'Social Sciences'))
print(s_type.shape)
print(s_type)

(41,)
['Liberal Arts' 'Liberal Arts' 'Liberal Arts' 'Liberal Arts' 'Sciences'
 'Sciences' 'Sciences' 'Sciences' 'Sciences' 'Sciences' 'Engineering'
 'Engineering' 'Engineering' 'Engineering' 'Engineering' 'Engineering'
 'Engineering' 'Engineering' 'Engineering' 'Engineering' 'Engineering'
 'Engineering' 'Engineering' 'Engineering' 'Engineering' 'Engineering'
 'Engineering' 'Engineering' 'Management' 'Management' 'Management'
 'Management' 'Management' 'Medicine' 'Medicine' 'Medicine' 'Medicine'
 'Medicine' 'Social Sciences' 'Social Sciences' 'Social Sciences']


In [4]:
df = df.dropna()
df.insert(loc=1, column='Type', value=s_type)
df.index = np.arange(1, len(df) + 1) # reset key (1-41)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 1 to 41
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   School                41 non-null     object 
 1   Type                  41 non-null     object 
 2   Department            41 non-null     object 
 3   Personnel             41 non-null     float64
 4   Expenses (unit:1000)  41 non-null     float64
 5   Space                 41 non-null     float64
 6   Teaching              41 non-null     float64
 7   Publications          41 non-null     float64
 8   Grants (unit:1000)    41 non-null     float64
dtypes: float64(6), object(3)
memory usage: 3.2+ KB


In [5]:
col_name = df.columns.tolist()
X = col_name[3:6]
Y = col_name[6:9]
print(f'X: {X}')
print(f'Y: {Y}')
df

X: ['Personnel', 'Expenses (unit:1000)', 'Space']
Y: ['Teaching', 'Publications', 'Grants (unit:1000)']


,School,Type,Department,Personnel,Expenses (unit:1000),Space,Teaching,Publications,Grants (unit:1000)
1,1,Liberal Arts,Chinese,33.5,6079.5,5376.0,26131.0,5.0,4250.9
2,2,Liberal Arts,Foreign Languages,36.5,8387.4,4574.0,40105.0,0.0,1543.5
3,3,Liberal Arts,History,26.0,5104.8,3795.0,13678.0,0.0,1729.1
4,4,Liberal Arts,Arts,6.0,3420.6,1610.0,4926.0,0.0,3562.2
5,5,Sciences,Mathematics,29.5,6426.2,4175.0,23647.0,27.0,10518.6
6,6,Sciences,Physics,44.8,9360.1,8184.0,27613.0,66.0,60912.4
7,7,Sciences,Chemistry,32.3,10099.4,9992.0,23419.0,49.0,36191.4
8,8,Sciences,Earth Sciences,16.0,5875.8,4128.0,7700.0,12.0,45590.3
9,9,Sciences,Biology,18.5,7136.1,5610.0,11981.0,13.5,19169.8
10,10,Sciences,Biotechnology,5.0,3123.2,1168.0,1710.0,6.5,14465.4


###  Overall Efficiency (OE, CRS Primal model)

In [6]:
# OE_crs = []
# for r in df['School']:
#     print(f'CRS_{r}')
#     lp = pulp.LpProblem(f"CRS_{r}", pulp.LpMaximize)
#     # Define Decicison Variables
#     var_key = range(1, df.shape[0] + 1)
#     U1 = pulp.LpVariable.dict("u1", var_key, lowBound=0)
#     U2 = pulp.LpVariable.dict("u2", var_key, lowBound=0)
#     U3 = pulp.LpVariable.dict("u3", var_key, lowBound=0)
#     V1 = pulp.LpVariable.dict("v1", var_key, lowBound=0)
#     V2 = pulp.LpVariable.dict("v2", var_key, lowBound=0)
#     V3 = pulp.LpVariable.dict("v3", var_key, lowBound=0)

#     # Define Objective Function
#     lp += U1[r]*df[Y[0]][r] + U2[r]*df[Y[1]][r] + U3[r]*df[Y[2]][r]

#     # Define Constraints
#     lp += V1[r]*df[X[0]][r] + V2[r]*df[X[1]][r] + V3[r]*df[X[2]][r] == 1
#     for k in var_key:
#         if r != k:
#             lp += U1[r]*df[X[0]][k] + U2[r]*df[X[1]][k] + U3[r]*df[X[2]][k] <= V1[r]*df[Y[0]][k] + V2[r]*df[Y[1]][k] + V3[r]*df[Y[2]][k]

#     # Solve Model
#     try:
#         s = lp.solve(GLPK_CMD(msg=False))
#         sol_dict = {1: 'Optimal', 2: 'Not Solved',
#                     3: 'Infeasible', 4: 'Unbounded', 5: 'Undefined'}
#         if 1 <= s <= 5:
#             print(f"({r}) Solver status:", sol_dict[s])
#     except Exception as e:
#         print("\n\nmodel.solve() has raised an ERROR:", e)
#     OE_crs.append(pulp.value(lp.objective))
#     print(f'U1 = {pulp.value(U1[r]):5<}, U2 = {pulp.value(U2[r]):5<}, U3 = {pulp.value(U3[r]):5<}')
#     print(f'V1 = {pulp.value(V1[r]):5<}, V2 = {pulp.value(V2[r]):5<}, V3 = {pulp.value(V3[r]):5<}')
#     # print(*df['Personnel'][r] + V2[r]*df['Expenses (unit:1000)'][r] + V3[r]*df['Space'][r])

# df['Efficiency (CRS)'] = OE_crs

### Overall Efficiency (CRS Dual Model)

In [7]:
OE_dual = []
Lamda_sum = []
Theta_crs = []
for r in range(1, df.shape[0]+1):
    print(f'CRS_Dual_{r}')
    lp = pulp.LpProblem(f"CRS_Dual_{r}", pulp.LpMinimize)
    # Define Decicison Variables
    var_key = range(1, df.shape[0]+1)
    TH = pulp.LpVariable(name="theta_r", lowBound=0)
    LM = pulp.LpVariable.dict("lamda", var_key, lowBound=0)

    # Define Objective Function
    lp += TH

    # Define Constraints
    for x in X:
        lp += pulp.lpSum(LM[k]*df[x][k] for k in var_key) <= TH*df[x][r]
    for y in Y:
        lp += pulp.lpSum(LM[k]*df[y][k] for k in var_key) >= df[y][r]

    # Solve Model
    try:
        s = lp.solve(GLPK_CMD(msg=False))
        sol_dict = {1: 'Optimal', 2: 'Not Solved',
                    3: 'Infeasible', 4: 'Unbounded', 5: 'Undefined'}
        if 1 <= s <= 5:
            print(f"({r}) Solver status:", sol_dict[s])
    except Exception as e:
        print("\n\nmodel.solve() has raised an ERROR:", e)
    OE_dual.append(pulp.value(lp.objective))
    Lamda_sum.append(pulp.value(pulp.lpSum(LM[k] for k in var_key)))
    Theta_crs.append(pulp.value(TH))

df['Overall Efficiency (CRS/Dual)'] = OE_dual
df['lamda_sum (CRS/Dual)'] = Lamda_sum
df['theta (CRS/Dual)'] = Theta_crs
VRS = ['IRS' if se < 1 else ('DRS' if se > 1 else 'CRS') for se in Lamda_sum]
df['VRS'] = VRS

CRS_Dual_1
(1) Solver status: Optimal
CRS_Dual_2
(2) Solver status: Optimal
CRS_Dual_3
(3) Solver status: Optimal
CRS_Dual_4
(4) Solver status: Optimal
CRS_Dual_5
(5) Solver status: Optimal
CRS_Dual_6
(6) Solver status: Optimal
CRS_Dual_7
(7) Solver status: Optimal
CRS_Dual_8
(8) Solver status: Optimal
CRS_Dual_9
(9) Solver status: Optimal
CRS_Dual_10
(10) Solver status: Optimal
CRS_Dual_11
(11) Solver status: Optimal
CRS_Dual_12
(12) Solver status: Optimal
CRS_Dual_13
(13) Solver status: Optimal
CRS_Dual_14
(14) Solver status: Optimal
CRS_Dual_15
(15) Solver status: Optimal
CRS_Dual_16
(16) Solver status: Optimal
CRS_Dual_17
(17) Solver status: Optimal
CRS_Dual_18
(18) Solver status: Optimal
CRS_Dual_19
(19) Solver status: Optimal
CRS_Dual_20
(20) Solver status: Optimal
CRS_Dual_21
(21) Solver status: Optimal
CRS_Dual_22
(22) Solver status: Optimal
CRS_Dual_23
(23) Solver status: Optimal
CRS_Dual_24
(24) Solver status: Optimal
CRS_Dual_25
(25) Solver status: Optimal
CRS_Dual_26
(26) S

In [8]:
df

,School,Type,Department,Personnel,Expenses (unit:1000),Space,Teaching,Publications,Grants (unit:1000),Overall Efficiency (CRS/Dual),lamda_sum (CRS/Dual),theta (CRS/Dual),VRS
1,1,Liberal Arts,Chinese,33.5,6079.5,5376.0,26131.0,5.0,4250.9,0.935939,0.657725,0.935939,IRS
2,2,Liberal Arts,Foreign Languages,36.5,8387.4,4574.0,40105.0,0.0,1543.5,1.000000,1.000000,1.000000,CRS
3,3,Liberal Arts,History,26.0,5104.8,3795.0,13678.0,0.0,1729.1,0.567983,0.345379,0.567983,IRS
4,4,Liberal Arts,Arts,6.0,3420.6,1610.0,4926.0,0.0,3562.2,0.770042,0.238425,0.770042,IRS
5,5,Sciences,Mathematics,29.5,6426.2,4175.0,23647.0,27.0,10518.6,1.000000,1.000000,1.000000,CRS
6,6,Sciences,Physics,44.8,9360.1,8184.0,27613.0,66.0,60912.4,1.000000,1.000000,1.000000,CRS
7,7,Sciences,Chemistry,32.3,10099.4,9992.0,23419.0,49.0,36191.4,0.804803,0.612000,0.804803,IRS
8,8,Sciences,Earth Sciences,16.0,5875.8,4128.0,7700.0,12.0,45590.3,0.717626,0.472454,0.717626,IRS
9,9,Sciences,Biology,18.5,7136.1,5610.0,11981.0,13.5,19169.8,0.665280,0.403758,0.665280,IRS
10,10,Sciences,Biotechnology,5.0,3123.2,1168.0,1710.0,6.5,14465.4,0.632991,0.141792,0.632991,IRS


### Technical Efficiency (VRS Model)

In [9]:
TE = []
VRS = []
for r in range(1, df.shape[0]+1):
    print(f'CRS_Duel_{r}')
    lp = pulp.LpProblem(f"CRS_Duel_{r}", pulp.LpMinimize)
    # Define Decicison Variables
    var_key = range(1, df.shape[0]+1)
    TH = pulp.LpVariable(name="theta_r", lowBound=0)
    LM = pulp.LpVariable.dict("lamda", var_key, lowBound=0)

    # Define Objective Function
    lp += TH

    # Define Constraints
    for x in X:
        lp += pulp.lpSum(LM[k]*df[x][k] for k in var_key) <= TH*df[x][r]
    for y in Y:
        lp += pulp.lpSum(LM[k]*df[y][k] for k in var_key) >= df[y][r]
    lp += pulp.lpSum(LM[k] for k in var_key) == 1

    # Solve Model
    try:
        s = lp.solve(GLPK_CMD(msg=False))
        sol_dict = {1: 'Optimal', 2: 'Not Solved',
                    3: 'Infeasible', 4: 'Unbounded', 5: 'Undefined'}
        if 1 <= s <= 5:
            print(f"({r}) Solver status:", sol_dict[s])
    except Exception as e:
        print("\n\nmodel.solve() has raised an ERROR:", e)
    # if 
    TE.append(pulp.value(TH))
    # VRS.append(pulp.value(LM[r]))

df['TE (VRS/Duel)'] = TE
df['SE'] = np.array(OE_dual) / np.array(TE)

CRS_Duel_1
(1) Solver status: Optimal
CRS_Duel_2
(2) Solver status: Optimal
CRS_Duel_3
(3) Solver status: Optimal
CRS_Duel_4
(4) Solver status: Optimal
CRS_Duel_5
(5) Solver status: Optimal
CRS_Duel_6
(6) Solver status: Optimal
CRS_Duel_7
(7) Solver status: Optimal
CRS_Duel_8
(8) Solver status: Optimal
CRS_Duel_9
(9) Solver status: Optimal
CRS_Duel_10
(10) Solver status: Optimal
CRS_Duel_11
(11) Solver status: Optimal
CRS_Duel_12
(12) Solver status: Optimal
CRS_Duel_13
(13) Solver status: Optimal
CRS_Duel_14
(14) Solver status: Optimal
CRS_Duel_15
(15) Solver status: Optimal
CRS_Duel_16
(16) Solver status: Optimal
CRS_Duel_17
(17) Solver status: Optimal
CRS_Duel_18
(18) Solver status: Optimal
CRS_Duel_19
(19) Solver status: Optimal
CRS_Duel_20
(20) Solver status: Optimal
CRS_Duel_21
(21) Solver status: Optimal
CRS_Duel_22
(22) Solver status: Optimal
CRS_Duel_23
(23) Solver status: Optimal
CRS_Duel_24
(24) Solver status: Optimal
CRS_Duel_25
(25) Solver status: Optimal
CRS_Duel_26
(26) S

In [10]:
df

,School,Type,Department,Personnel,Expenses (unit:1000),Space,Teaching,Publications,Grants (unit:1000),Overall Efficiency (CRS/Dual),lamda_sum (CRS/Dual),theta (CRS/Dual),VRS,TE (VRS/Duel),SE
1,1,Liberal Arts,Chinese,33.5,6079.5,5376.0,26131.0,5.0,4250.9,0.935939,0.657725,0.935939,IRS,0.968282,0.966598
2,2,Liberal Arts,Foreign Languages,36.5,8387.4,4574.0,40105.0,0.0,1543.5,1.000000,1.000000,1.000000,CRS,1.000000,1.000000
3,3,Liberal Arts,History,26.0,5104.8,3795.0,13678.0,0.0,1729.1,0.567983,0.345379,0.567983,IRS,0.660915,0.859389
4,4,Liberal Arts,Arts,6.0,3420.6,1610.0,4926.0,0.0,3562.2,0.770042,0.238425,0.770042,IRS,1.000000,0.770042
5,5,Sciences,Mathematics,29.5,6426.2,4175.0,23647.0,27.0,10518.6,1.000000,1.000000,1.000000,CRS,1.000000,1.000000
6,6,Sciences,Physics,44.8,9360.1,8184.0,27613.0,66.0,60912.4,1.000000,1.000000,1.000000,CRS,1.000000,1.000000
7,7,Sciences,Chemistry,32.3,10099.4,9992.0,23419.0,49.0,36191.4,0.804803,0.612000,0.804803,IRS,0.811823,0.991353
8,8,Sciences,Earth Sciences,16.0,5875.8,4128.0,7700.0,12.0,45590.3,0.717626,0.472454,0.717626,IRS,0.791274,0.906925
9,9,Sciences,Biology,18.5,7136.1,5610.0,11981.0,13.5,19169.8,0.665280,0.403758,0.665280,IRS,0.711658,0.934831
10,10,Sciences,Biotechnology,5.0,3123.2,1168.0,1710.0,6.5,14465.4,0.632991,0.141792,0.632991,IRS,1.000000,0.632991


In [11]:
df.to_excel('DEA_Result.xlsx')

### Analysis (Department)

In [12]:
sorter_VRS = ['DRS', 'IRS', 'CRS']
sorterIndex_VRS = dict(zip(sorter_VRS, range(len(sorter_VRS))))
df['VRS_rank'] = df['VRS'].map(sorterIndex_VRS)

sorter_Type = ['Liberal Arts','Sciences','Engineering','Management','Medicine','Social Sciences']
sorterIndex_Type = dict(zip(sorter_Type, range(len(sorter_Type))))
df['Type_rank'] = df['Type'].map(sorterIndex_Type)

In [13]:
df.sort_values(['VRS_rank', 'Type_rank'], ascending=[True, True])[['VRS', 'Type', 'Department', 'Overall Efficiency (CRS/Dual)', 'TE (VRS/Duel)', 'SE']]

,VRS,Type,Department,Overall Efficiency (CRS/Dual),TE (VRS/Duel),SE
34,DRS,Medicine,Medicine,0.963368,1.000000,0.963368
1,IRS,Liberal Arts,Chinese,0.935939,0.968282,0.966598
3,IRS,Liberal Arts,History,0.567983,0.660915,0.859389
4,IRS,Liberal Arts,Arts,0.770042,1.000000,0.770042
7,IRS,Sciences,Chemistry,0.804803,0.811823,0.991353
8,IRS,Sciences,Earth Sciences,0.717626,0.791274,0.906925
9,IRS,Sciences,Biology,0.665280,0.711658,0.934831
10,IRS,Sciences,Biotechnology,0.632991,1.000000,0.632991
11,IRS,Engineering,Mechanical Eng.,0.852982,0.854853,0.997811
13,IRS,Engineering,Information Eng.,0.962564,0.981559,0.980648


In [33]:
stat_school = df.groupby('Type')[['Overall Efficiency (CRS/Dual)', 'TE (VRS/Duel)', 'SE']].mean().sort_values('Overall Efficiency (CRS/Dual)')
stat_school.to_excel('stat_school.xlsx')

In [23]:
group = df.groupby(['Type', 'VRS']).count()['School']
group

Type             VRS
Engineering      CRS     7
                 IRS    11
Liberal Arts     CRS     1
                 IRS     3
Management       CRS     2
                 IRS     3
Medicine         DRS     1
                 IRS     4
Sciences         CRS     2
                 IRS     4
Social Sciences  CRS     1
                 IRS     2
Name: School, dtype: int64

In [30]:
print(group[5])

3
